In [17]:
import pandas as pd
import h5py
import os
import h5py
import os

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import cv2
import numpy as np

# Mobilenet modeli sadece feature çıkarımı için
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
feature_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

/var/folders/kw/3t8qd9fd6hlfbqdps6pvpchw0000gn/T/ipykernel_90893/841802844.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


In [4]:

X = []
y = []

for file in os.listdir("gestureData/"):
    if file.endswith(".mat"):
        user_id = int(file.split("_")[2])
        with h5py.File(f"gestureData/{file}", "r") as f:
            key = list(f.keys())[0]  # genellikle 'depth_sequence'
            sequence = f[key][:]  # [frames, height, width]
            
            # Sadece ilk 10 kare, her kareyi 64x64'e yeniden boyutlandır
            if sequence.shape[0] < 10:
                continue
            
            # frames = sequence[:10]
            # resized = [cv2.resize(frame, (64, 64)) for frame in frames]
            # stacked = np.stack(resized, axis=0)  # [25, 64, 64]

            frame_features = []
            for frame in sequence:
                resized = cv2.resize(frame, (64, 64))
                if len(resized.shape) == 2:
                    resized = np.stack([resized]*3, axis=-1)  # grayscale → RGB
                resized = resized.astype(np.float32) / 255.0
                feat = feature_model.predict(resized[np.newaxis, ...], verbose=0)
                frame_features.append(feat[0])
            
            frame_features = np.array(frame_features)  # [num_frames, 1280]
            mean = np.mean(frame_features, axis=0)
            std = np.std(frame_features, axis=0)
            concatenated = np.concatenate([mean, std])  # [2560]

            X.append(concatenated)
            y.append(user_id)


In [ ]:
X = np.array(X)  # shape: [num_samples, 10, 64, 64]
y = np.array(y)

# CNN için channel ekle (örneğin grayscale için kanal sayısı = 1)
X = X[..., np.newaxis]  # -> [num_samples, 10, 64, 64, 1]
X = X.reshape(333, 2560)

In [8]:
len(y)

333

In [18]:
# y zaten elinde numpy array olarak var
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))  # 10 sınıf
y_cat = to_categorical(y_encoded, num_classes=num_classes)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Eğitim/test böl
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(2560,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, epochs=35, batch_size=16, validation_data=(X_test, y_test))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 512)            │     1,311,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,445,130 (5.51 MB)

 Trainable params: 1,445,130 (5.51 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1025 - loss: 2.8288 - val_accuracy: 0.1194 - val_loss: 2.4917
Epoch 2/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1779 - loss: 2.4694 - val_accuracy: 0.1194 - val_loss: 2.4391
Epoch 3/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2582 - loss: 2.2165 - val_accuracy: 0.1791 - val_loss: 2.1985
Epoch 4/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2839 - loss: 2.0256 - val_accuracy: 0.1940 - val_loss: 2.2301
Epoch 5/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2988 - loss: 2.0179 - val_accuracy: 0.1940 - val_loss: 2.1644
Epoch 6/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3821 - loss: 1.7561 - val_accuracy: 0.2537 - val_loss: 2.2127
Epoch 7/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3984 - loss: 1.7479 - val_accuracy: 0.2687 - val_loss: 2.0746
Epoch 8/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4348 - loss: 1.6341 - val_accuracy: 0.2388 - val_loss

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_simple, y_test_simple = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train_simple)

y_pred = clf.predict(X_test)
print(classification_report(y_test_simple, y_pred))

# Accuracy
acc = accuracy_score(y_test_simple, y_pred)
print(f"✅ Accuracy: {acc:.4f}")

              precision    recall  f1-score   support

           1       0.40      0.50      0.44         4
           2       0.33      0.40      0.36         5
           3       0.00      0.00      0.00         5
           4       0.56      0.83      0.67         6
           5       0.56      0.62      0.59         8
           6       0.00      0.00      0.00         3
           7       0.67      0.55      0.60        11
           8       0.60      0.33      0.43         9
           9       0.78      1.00      0.88         7
          10       0.55      0.67      0.60         9

    accuracy                           0.54        67
   macro avg       0.44      0.49      0.46        67
weighted avg       0.51      0.54      0.51        67

✅ Accuracy: 0.5373
